In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/boot/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: HIP initialization: Unexpected error from hipGetDeviceCount(). Did you run some cuda functions before calling NumHipDevices() that might have already set an error? Error 101: hipErrorInvalidDevice (Triggered internally at ../c10/hip/HIPFunctions.cpp:110.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# hyperparameters
input_size = 28 * 28
hidden_size = 100
num_of_classes = 10   # MNIST digits
num_epochs = 2
batch_size = 32
learning_rate = 0.001

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [6]:
# CIFAR10
train_dataset = torchvision.datasets.CIFAR10(root="./data/", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root="./data/", train=False, transform=transform, download=False)

# Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data/


In [7]:
examples = iter(train_loader)
samples, labels = next(examples)
print(samples.shape, labels.shape)

torch.Size([32, 3, 32, 32]) torch.Size([32])
